In [2]:
import pandas as pd
import os, sys
os.chdir("../")
sys.path.append('./src')
from utils.get_concept_subsets import SUBSETS

In [3]:
dfs = []
directories = ['./results/experiment_2/raw_results/gpt4_boolean_translated', 
              './results/experiment_2/raw_results/gpt4_fol_translated']

for directory in directories:
    for file in os.listdir(directory):
        if file.endswith('.csv'):
            df = pd.read_csv(os.path.join(directory, file), index_col=0)
            df['consistency'] = df['rule_evaluation'].astype(str) == df['model_answer'].astype(str)

            current_set = 0
            sets = []
            for i, row in df.iterrows():
                if df['model_reply'].iloc[i] != df['model_reply'].iloc[i-1]:
                    current_set += 1
                sets.append(current_set)
            df['set'] = sets

            df.to_csv(os.path.join(directory, file))
            dfs.append(df)

mdf = pd.concat(dfs)

In [3]:
mdf.to_csv('/users/aloo1/thesis/calibration-rule-learning/results/experiment_2/compiled_all_rules.csv')

In [31]:
mdf[(mdf['set'] >= 3) & (~mdf['concept_num'].isin(SUBSETS['boolean']))]['consistency'].mean()

0.8917629792698587

In [4]:
bconcepts = mdf[(mdf['set'] >= 3) & (mdf['concept_num'].isin(SUBSETS['boolean']))]

bdf = bconcepts[bconcepts['set'] == 25].drop_duplicates(['model_reply']).loc[:, ['rule_match', 'rule_coverage', 'concept_num', 'concept', 'rule_expression', 'model_reply', 'code']].sort_values(['rule_match'])
bdf.to_csv('./results/experiment_2/viewer_boolean_rules.csv')
bdf

,rule_match,rule_coverage,concept_num,concept,rule_expression,model_reply,code
66,False,0.818182,hg18,large,large and not circle,"Based on your examples, I've learned that the ...",def is_rule(obj):\n return obj.large and no...
68,False,0.779412,hg84,(circle or blue) or (triangle and green),blue or green,"Based on your examples, I've learned that the ...",def is_rule(obj):\n return obj.blue or obj....
76,False,0.763158,hg83,(circle and blue) or (triangle and green),(circle or triangle) and (blue or green) and n...,"Based on your examples, I've learned that the ...",def is_rule(obj):\n return (obj.circle or o...
60,False,0.483333,hg82,(circle or triangle) implies blue,False,"Based on your examples, I've learned that the ...",def is_rule(obj):\n return False
61,False,0.819672,hg81,blue implies size 1,or ( ),"Based on your examples, I've learned that the ...",def is_rule(obj):\n return obj.color or (ob...
73,False,1.000000,hg80,(not blue) implies (not circle),(not circle) or (circle and blue) or triangle ...,"Based on your examples, I've learned that the ...",def is_rule(obj):\n return (not obj.circle)...
83,False,0.710843,hg79,(not blue) implies circle,(blue or green) and (circle),"Based on your examples, I've learned that the ...",def is_rule(obj):\n return (obj.blue or obj...
89,False,0.932584,hg78,blue implies circle,not(blue) and not(triangle and blue),"Based on your examples, I've learned that the ...",def is_rule(obj):\n return not(obj.blue) an...
66,False,0.939394,hg77,circle implies blue,rectangle or triangle,"Based on your examples, I've learned that the ...",def is_rule(obj):\n return obj.rectangle or...
67,False,0.701493,hg25,blue or small,small and (triangle or rectangle),"Based on your examples, I've learned that the ...",def is_rule(obj):\n return obj.small and (o...


In [21]:
from src.translate_explanations import evaluate_python_code, process_python_code
import numpy as np
import math
def check_rule_coverage(annotated_df, object_df):
    hdf = pd.read_csv('./results/experiment_1/human_concept_stats.csv')
    coverage = []
    chance = []
    human_lq = []
    expression = []
    for i, row in annotated_df.iterrows():
        objs = object_df[(object_df['concept_num'] == row['concept_num']) & (object_df['set'] < 25)]['object']
        answers = object_df[(object_df['concept_num'] == row['concept_num']) & (object_df['set'] < 25)]['answer']
        yesrate = object_df[(object_df['concept_num'] == row['concept_num']) & (object_df['set'] < 25)]['answer'].mean()
        chance.append((yesrate * yesrate) + ((1-yesrate) * (1-yesrate)))
        human_lq.append(hdf[hdf['concept'] == row['concept_num']]['last_quarter_mean'].iloc[0])

        objs = objs[(math.floor(len(objs) * 3/4)):]
        answers = answers[(math.floor(len(answers) * 3/4)):]
        
        coverage.append((np.array([evaluate_python_code(row['manual_code'], o) for o in objs]) == answers).mean())
        expression.append(process_python_code(row['manual_code']))

    annotated_df['rule_expression'] = expression
    annotated_df['human_lq'] = human_lq
    annotated_df['chance'] = chance
    annotated_df['manual_rule_coverage'] = coverage
    annotated_df['chance_difference'] = np.array(coverage) - np.array(chance)
    annotated_df['human_lq_diff'] = np.array(coverage) - np.array(human_lq)

    return annotated_df

In [22]:
manual_rule_coverage = check_rule_coverage(pd.read_csv('./results/experiment_2/viewer_boolean_rules_manualcoded.csv'), mdf).sort_values(['manual_rule_coverage'])
manual_rule_coverage.loc[:, ['concept_num', 'manual_rule_coverage', 'human_lq', 'concept', 'rule_expression']]

,concept_num,manual_rule_coverage,human_lq,concept,rule_expression
17,hg15,0.526316,0.730159,not (circle xor blue),green or (triangle)
20,hg16,0.588235,0.745455,circle xor (not blue),not (medium and blue) and not (large and blue)
21,hg09,0.684211,0.934783,circle and blue,large and blue
12,hg21,0.700000,0.855769,large or small,large or ((blue or green) and not (medium and ...
2,hg83,0.736842,0.851240,(circle and blue) or (triangle and green),(circle or triangle) and (blue or green) and n...
9,hg25,0.764706,0.893939,blue or small,small and (triangle or rectangle)
1,hg84,0.764706,0.842857,(circle or blue) or (triangle and green),blue or green
0,hg18,0.764706,0.956667,large,large and not circle
26,hg14,0.777778,0.804348,circle xor blue,(circle and ((large and green) or (medium and ...
27,hg11,0.789474,0.952174,not (circle and blue),not (medium and blue)


In [32]:
fmanual_rule_coverage = check_rule_coverage(pd.read_csv('./results/experiment_2/viewer_fol_rules_manualcoded.csv'), mdf).sort_values(['manual_rule_coverage'])
fmanual_rule_coverage.loc[:, ['concept_num', 'manual_rule_coverage', 'human_lq', 'chance', 'concept', 'rule_expression']].iloc[:30]

,concept_num,manual_rule_coverage,human_lq,chance,concept,rule_expression
75,hg91,0.318182,0.867424,0.796539,same shape as an object that is blue or green,not (large and yellow and circle) and not (med...
58,hg111,0.363636,0.577640,0.500623,at least one other object of the same shape is...,circle or rectangle
57,hg104,0.388889,0.676364,0.508034,exactly one other object is the same color,yellow or blue and (rectangle or circle or tri...
66,hg112,0.388889,0.764000,0.517748,at least one other object of the same shape is...,(blue and (triangle or rectangle)) or (yellow ...
68,hg75,0.388889,0.652000,0.503673,there is another object of the same shape<br>t...,((small or medium) and (yellow or blue) and (c...
65,hg74,0.409091,0.590062,0.507993,there is another object of the same shape<br>a...,small or (triangle and (medium or large) and n...
71,hg26,0.421053,0.831169,0.791205,one of the largest or smallest,(large and green) or (large and circle) or (gr...
64,hg63,0.473684,0.823333,0.505540,same color as another object,(triangle or rectangle) and (medium or large)
61,hg45,0.500000,0.663043,0.553002,same shape as one of the largest,(triangle or circle) and (green or yellow or b...
60,hg90,0.500000,0.614719,0.514694,same shape as another object that is blue or g...,(large and rectangle) or (small and (blue or y...


In [40]:
import plotly.express as px
px.box(manual_rule_coverage, y=['human_lq_diff', 'chance_difference'], points='all', hover_data='concept', width=500)

In [18]:
fconcepts = mdf[(mdf['set'] >= 3) & (~mdf['concept_num'].isin(SUBSETS['boolean']))]

fdf = fconcepts[fconcepts['set'] == 25].drop_duplicates(['model_reply']).loc[:, ['rule_match', 'rule_coverage', 'concept_num', 'concept', 'rule_expression', 'model_reply', 'code']].sort_values(['rule_match'])
fdf.to_csv('./results/experiment_2/viewer_fol_rules.csv')
fdf

,rule_match,rule_coverage,concept_num,concept,rule_expression,model_reply,code
69,False,0.753623,hg100,one of the smallest of objects who share a sha...,(not large) or (yellow and triangle),"Based on your examples, I've learned that the ...",def is_rule(obj):\n return (not obj.large) ...
75,False,0.680000,hg67,blue or (same color as all other objects of th...,blue or (triangle) or (green and circle) or (y...,"Based on your examples, I've learned that the ...",def is_rule(obj):\n return obj.blue or (obj...
60,False,0.650000,hg66,same color as all other objects of the same shape,(blue or yellow) and (large or triangle or rec...,"Based on your examples, I've learned that the ...",def is_rule(obj):\n return (obj.blue or obj...
68,False,0.632353,hg65,same shape and color as another object,(medium and circle) or (rectangle and not yell...,"Based on your examples, I've learned that the ...",def is_rule(obj):\n return (obj.medium and ...
77,False,0.389610,hg64,no other object has the same shape,and,"Based on your examples, I've learned that the ...",def is_rule(obj):\n return obj.size and obj...
...,...,...,...,...,...,...,...
63,False,0.698413,hg35,larger than some other object,(large and blue) or (medium and yellow) or ((t...,"Based on your examples, I've learned that the ...",def is_rule(obj):\n return (obj.large and o...
64,False,0.906250,hg34,blue or (larger or equal in size to all other ...,large or blue,"Based on your examples, I've learned that the ...",def is_rule(obj):\n return obj.large or obj...
68,False,0.955882,hg33,one of the largest and blue,large and blue,"Based on your examples, I've learned that the ...",def is_rule(obj):\n return obj.large and ob...
75,False,0.720000,hg40,(not blue) and same shape as a blue object,(small and circle) or (small and rectangle) an...,"Based on your examples, I've learned that the ...",def is_rule(obj):\n return (obj.small and o...
